In [3]:
import numpy as np
import torch
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm.notebook import tqdm
from typing import Optional
import sys

/home/sasha/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
sys.path.insert(0, "../")
print(sys.path)

['../', '/home/sasha/BMM/bayes_deep_compression/examples', '/home/sasha/anaconda3/lib/python311.zip', '/home/sasha/anaconda3/lib/python3.11', '/home/sasha/anaconda3/lib/python3.11/lib-dynload', '', '/home/sasha/anaconda3/lib/python3.11/site-packages']


In [8]:

class Classifier(nn.Module): 
    def __init__(self, classes: int = 10): 
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 
        self.pool = nn.MaxPool2d(2, 2) 
        #self.dropout1 = nn.Dropout2d(0.25) 
        #self.dropout2 = nn.Dropout2d(0.5) 
        self.fc1 = nn.Linear(64 * 7 * 7, 128) 
        self.fc2 = nn.Linear(128, classes) 
  
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x))) 
        #x = self.dropout1(x) 
        x = self.pool(F.relu(self.conv2(x))) 
        #x = self.dropout2(x) 
        x = x.view(-1, 64 * 7 * 7) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return x

In [5]:
from src.methods.bayes.base.distribution import ParamDist

In [9]:
from src.methods.bayes.variational.net import VarBayesModuleNet
from src.methods.bayes.variational.net import NormalVarBayesModule
from src.methods.bayes.variational.optimization import VarRenuiLoss

In [10]:
from src.methods.bayes.variational.trainer import VarBayesTrainer, VarTrainerParams, Beta_Scheduler_Plato, CallbackLossAccuracy
from src.methods.bayes.variational.trainer import Beta_Scheduler
from src.methods.report.base import ReportChain
from src.methods.report.variational import VarBaseReport

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
test_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
BATCH_SIZE=1000
EPOCHS=1000
LR = 1e-3 #5e-4
# Split the training set into training and validation sets 
VAL_PERCENT = 0.2 # percentage of the data used for validation 
SAMPLES = 10
BETA = 1e-2 #5e-5 #len(train_dataset) *1. / BATCH_SIZE
BETA_FAC = 5e-1
PRUNE = -5#1.99, 2.1
PLATO_TOL = 20

base_module = Classifier()
var_module = NormalVarBayesModule(base_module)
model = VarBayesModuleNet(base_module, nn.ModuleList([var_module]))


optimizer = optim.Adam(model.parameters(), lr=LR)

fit_loss = nn.CrossEntropyLoss(reduction="sum") 
kl_loss = VarRenuiLoss()
beta = BETA#len(train_dataset) *1. / BATCH_SIZE
print(beta)
#beta = Beta_Scheduler(beta=(len(train_dataset) *1. / BATCH_SIZE))

# beta_KL = Beta_Scheduler_Plato(beta.beta, 1 / BETA_FAC, PLATO_TOL, ref = beta, threshold=1e-4)

#Данная функция будет выполнятся после каждого шага тренера, соответсвенно нам требуется сделать шаг планировщика и изменить соотвествующий коэффициент
    
#print(model.base_module.state_dict().keys())
val_size    = int(VAL_PERCENT * len(train_dataset)) 
train_size  = len(train_dataset) - val_size 

t_dataset, v_dataset = torch.utils.data.random_split(train_dataset,  
                                                        [train_size,  
                                                            val_size]) 

# Create DataLoaders for the training and validation sets 
train_loader = torch.utils.data.DataLoader(t_dataset,  
                                        batch_size=BATCH_SIZE,  
                                        shuffle=True, 
                                        pin_memory=True) 

eval_loader = torch.utils.data.DataLoader(v_dataset,  
                                        batch_size=BATCH_SIZE,  
                                        shuffle=False, 
                                        pin_memory=True)

model.to(device) 
train_params = VarTrainerParams(EPOCHS, optimizer,fit_loss, kl_loss, SAMPLES, PRUNE, beta, {'accuracy': CallbackLossAccuracy()})
#trainer = VarBayesTrainer(train_params, ReportChain([VarBaseReport()]), train_loader, eval_loader, [post_train_step])
trainer = VarBayesTrainer(train_params, ReportChain([VarBaseReport()]), train_loader, eval_loader)
trainer.train(model)

0.01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch [1/1000],Loss:4222.001953125, KL Loss: 185763.734375. FitLoss: 2539.72900390625,Accuracy:0.10428750000000002,Validation Loss:4147.40283203125,Validation Accuracy:0.112, Prune parameters: 606.0/421642,Beta: 0.01
Epoch [2/1000],Loss:4173.61181640625, KL Loss: 185788.125. FitLoss: 2426.019775390625,Accuracy:0.11676250000000002,Validation Loss:4130.67431640625,Validation Accuracy:0.182, Prune parameters: 648.0/421642,Beta: 0.01
Epoch [3/1000],Loss:4140.1572265625, KL Loss: 185946.3125. FitLoss: 2343.304931640625,Accuracy:0.127375,Validation Loss:4116.85107421875,Validation Accuracy:0.15, Prune parameters: 663.0/421642,Beta: 0.01
Epoch [4/1000],Loss:4070.2421875, KL Loss: 186088.1875. FitLoss: 2271.291015625,Accuracy:0.1582125,Validation Loss:4074.47607421875,Validation Accuracy:0.357, Prune parameters: 649.0/421642,Beta: 0.01
Epoch [5/1000],Loss:4011.5830078125, KL Loss: 186233.9375. FitLoss: 2230.14208984375,Accuracy:0.18063750000000003,Validation Loss:3951.05517578125,Validation Ac